In [1]:
######## CLASSIFICATION OF SENTENCESS BASED ON Universal-sentence-encoder and keras modelling #########
## https://www.dlology.com/blog/keras-meets-universal-sentence-encoder-transfer-learning-for-text-data/

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
# import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
# import seaborn as sns

from tensorflow import keras
from keras.layers import Input, Embedding, LSTM, Dense, Lambda
from keras.models import Model
import numpy as np

Using TensorFlow backend.


In [2]:
def get_dataframe(filename):
    lines = open(filename, 'r').read().splitlines()
    data = []
    for i in range(0, len(lines)):
        label = lines[i].split(' ')[0]
        label = label.split(",")[0]
        text = ' '.join(lines[i].split(',')[1:])
        text = re.sub(',','', text)
        data.append([label, text])

    df = pd.DataFrame(data, columns=['label', 'text'])
    df.label = df.label.astype('category')
    return df

df_train = get_dataframe('sentences.txt')
df_train.head()

,label,text
0,0,I need daily rations in Attur
1,0,Give rice dal oil
2,0,people are hungary
3,0,provide rations for 6 people for 10 days
4,0,Need prepared food for 20 people in slum


In [460]:
# df_train

In [461]:
### list of one-hot encoded labels
train_text = df_train['text'].tolist()
train_text = np.array(train_text, dtype=object)[:, np.newaxis]

train_label = np.asarray(pd.get_dummies(df_train.label), dtype = np.int8)

In [462]:
category_counts = len(df_train.label.cat.categories)
module_url = "tf_sent_encoder_2_MODEL_PRETRAINED" 
embed = hub.Module(module_url)
embed_size = embed.get_output_info_dict()['default'].get_shape()[1].value

In [340]:
def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)),signature="default", as_dict=True)["default"]


In [8]:

input_text = Input(shape=(1,), dtype=tf.string)
embedding = Lambda(UniversalEmbedding,output_shape=(embed_size,))(input_text)
dense1 = Dense(256, activation='relu')(embedding)
dense2 = Dense(128, activation='relu')(dense1)
pred = Dense(category_counts, activation='softmax')(dense2)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

W0504 14:34:07.140520 20892 deprecation_wrapper.py:119] From c:\users\i511977\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0504 14:34:07.143542 20892 deprecation_wrapper.py:119] From c:\users\i511977\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0504 14:34:09.596715 20892 deprecation_wrapper.py:119] From c:\users\i511977\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0504 14:34:09.793557 20892 deprecation_wrapper.py:119] From c:\users\i511977\appdata\local\programs\python\python37\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 645       
Total params: 164,869
Trainable params: 164,869
Non-trainable params: 0
_________________________________________________________________


In [388]:
df_test = get_dataframe('test.txt')
### list of one-hot encoded labels
test_text = df_test['text'].tolist()
test_text = np.array(test_text, dtype=object)[:, np.newaxis]

test_label = np.asarray(pd.get_dummies(df_test.label), dtype = np.int8)

In [390]:
test_text

array([['Need food for diabeties patient'],
       ['I want bread and rice for my home'],
       ['arm broken need to go to hospital'],
       ['food poisoning by eating old distributed food'],
       ['network tower broken down in whitefield'],
       ['calls are not connecting from airtel'],
       ['I am a doctor  my car brokedown and want it to get repaired urgently.'],
       ['My visa about to get expire. Want some travel arrangements to be done to fly back to Austia.'],
       ['My neighbor attempted suicide .She is in hospital right now  urgent help needed'],
       ['I saw bruises on my maid face. I think she is a victim of domestic violence.']],
      dtype=object)

In [21]:
from keras import backend as K

with tf.Session() as session:
  K.set_session(session)
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  history = model.fit(train_text, 
            train_label,
            validation_data=(test_text, test_label),
            epochs=10,batch_size=20)
  model.save_weights('./model.h5')

Train on 40 samples, validate on 10 samples
Epoch 1/10
20/40 [==============>...............] - ETA: 2s - loss: 1.6075 - acc: 0.2000

Exception ignored in: <function BaseSession._Callable.__del__ at 0x000001BD7E89D828>
Traceback (most recent call last):
  File "c:\users\i511977\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\client\session.py", line 1473, in __del__
    self._session._session, self._handle)
tensorflow.python.framework.errors_impl.CancelledError: (None, None, 'Session has been closed.')


40/40 [==============================] - 4s 88ms/step - loss: 1.6110 - acc: 0.1500 - val_loss: 1.5837 - val_acc: 0.4000
Epoch 2/10
40/40 [==============================] - 0s 2ms/step - loss: 1.5484 - acc: 0.7250 - val_loss: 1.5432 - val_acc: 0.6000
Epoch 3/10
40/40 [==============================] - 0s 2ms/step - loss: 1.4864 - acc: 0.9500 - val_loss: 1.4984 - val_acc: 0.7000
Epoch 4/10
20/40 [==============>...............] - ETA: 0s - loss: 1.4171 - acc: 0.9500

Exception ignored in: <function BaseSession._Callable.__del__ at 0x000001BD7E89D828>
Traceback (most recent call last):
  File "c:\users\i511977\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\client\session.py", line 1473, in __del__
    self._session._session, self._handle)
tensorflow.python.framework.errors_impl.CancelledError: (None, None, 'Session has been closed.')


40/40 [==============================] - 0s 2ms/step - loss: 1.4202 - acc: 0.9750 - val_loss: 1.4440 - val_acc: 0.7000
Epoch 5/10
40/40 [==============================] - 0s 1ms/step - loss: 1.3395 - acc: 0.9750 - val_loss: 1.3813 - val_acc: 0.7000
Epoch 6/10
40/40 [==============================] - 0s 2ms/step - loss: 1.2457 - acc: 1.0000 - val_loss: 1.3108 - val_acc: 0.7000
Epoch 7/10
40/40 [==============================] - 0s 2ms/step - loss: 1.1359 - acc: 1.0000 - val_loss: 1.2333 - val_acc: 0.7000
Epoch 8/10
40/40 [==============================] - 0s 2ms/step - loss: 1.0181 - acc: 1.0000 - val_loss: 1.1511 - val_acc: 0.7000
Epoch 9/10
40/40 [==============================] - 0s 1ms/step - loss: 0.8930 - acc: 1.0000 - val_loss: 1.0709 - val_acc: 0.7000
Epoch 10/10
40/40 [==============================] - 0s 2ms/step - loss: 0.7621 - acc: 1.0000 - val_loss: 0.9899 - val_acc: 0.7000


In [22]:
new_text = ["wheat exhausted in hospital canteen", 
            "my neighbors house has crying voices. I think they need help"]

new_text = np.array(new_text, dtype=object)[:, np.newaxis]
with tf.Session() as session:
  K.set_session(session)
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  model.load_weights('./model.h5')  
  predicts = model.predict(new_text)

categories = df_train.label.cat.categories.tolist()
predict_logits = predicts.argmax(axis=1)
predict_labels = [categories[logit] for logit in predict_logits]
print(predict_labels)

Exception ignored in: <function BaseSession._Callable.__del__ at 0x000001BD7E89D828>
Traceback (most recent call last):
  File "c:\users\i511977\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\client\session.py", line 1473, in __del__
    self._session._session, self._handle)
tensorflow.python.framework.errors_impl.CancelledError: (None, None, 'Session has been closed.')


['0', '4']


In [23]:
predicts

array([[0.4154314 , 0.2754173 , 0.07836786, 0.13586688, 0.09491654],
       [0.16206545, 0.1603227 , 0.15349114, 0.09350415, 0.43061662]],
      dtype=float32)

In [118]:
###################################################################################################################

In [ ]:
########## With LARGE augumented datasets -  Single Label - Universal Sentense Encoder - Keras modelling ##########

In [120]:
import tensorflow as tf
import tensorflow_hub as hub
# import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

from tensorflow import keras
from keras.layers import Input, Embedding, LSTM, Dense, Lambda
from keras.models import Model
import numpy as np

In [609]:
from sklearn.model_selection import train_test_split
from sklearn import datasets
import pandas as pd

df = pd.DataFrame(columns=['label','text'])
trainset = sklearn.datasets.load_files(container_path = 'datalarge',encoding = 'UTF-8')

def get_dataframe(lines):
#     lines = filename.splitlines()
    data = []
    for i in range(0, len(lines)):
        label = lines[i][0]
    #     label = label.split(",")[0]
        text = ' '.join(lines[i][1:])
        text = re.sub(',','', text)
        data.append([label, text])
    df = pd.DataFrame(data, columns=['label', 'text'])
    df.label = df.label.astype('category')
    return df

for k in range(len(trainset.target)):
    labelled_sent= trainset.data[k].strip().split('\r\n')
    Label=[trainset.target[k]]*len(labelled_sent)
    labelled_emb = list(zip([s for s in Label], [s for s in labelled_sent]))
    df_train = get_dataframe(labelled_emb)
    df=pd.concat([df,df_train])

In [579]:
X_train, X_test, y_train, y_test = train_test_split(df, df['label'], test_size=0.2,random_state=42)
X_train.reset_index(inplace=True, drop=True)
X_test.reset_index(inplace=True, drop=True)
y_train.reset_index(inplace=True, drop=True)
y_test.reset_index(inplace=True, drop=True)


In [477]:
train_text = np.array(X_train, dtype=object)[:,1][:,np.newaxis]
train_label = np.asarray(pd.get_dummies(X_train.label), dtype = np.int8)

In [478]:
test_text = np.array(X_test, dtype=object)[:,1][:,np.newaxis]
test_label = np.asarray(pd.get_dummies(X_test.label), dtype = np.int8)

In [481]:
category_counts = len(np.unique(X_train.label))
module_url = "tf_sent_encoder_2_MODEL_PRETRAINED" 
embed = hub.Module(module_url)
embed_size = embed.get_output_info_dict()['default'].get_shape()[1].value

In [482]:
def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)),signature="default", as_dict=True)["default"]


In [483]:

input_text = Input(shape=(1,), dtype=tf.string)
embedding = Lambda(UniversalEmbedding,output_shape=(embed_size,))(input_text)
dense1 = Dense(256, activation='relu')(embedding)
dense2 = Dense(128, activation='relu')(dense1)
pred = Dense(category_counts, activation='softmax')(dense2)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

In [486]:
from keras import backend as K

with tf.Session() as session:
  K.set_session(session)
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  history = model.fit(train_text, 
            train_label,
            validation_data=(test_text, test_label),epochs=5,batch_size=50)
  model.save_weights('./model.h5')

Train on 3540 samples, validate on 885 samples
Epoch 1/5
3540/3540 [==============================] - 264s 75ms/step - loss: 0.8755 - acc: 0.6969 - val_loss: 0.2986 - val_acc: 0.9051
Epoch 2/5
3540/3540 [==============================] - 15s 4ms/step - loss: 0.2148 - acc: 0.9325 - val_loss: 0.1674 - val_acc: 0.9401
Epoch 3/5
3540/3540 [==============================] - 15s 4ms/step - loss: 0.1162 - acc: 0.9664 - val_loss: 0.1158 - val_acc: 0.9616
Epoch 4/5
3540/3540 [==============================] - 15s 4ms/step - loss: 0.0733 - acc: 0.9828 - val_loss: 0.0951 - val_acc: 0.9661
Epoch 5/5
3540/3540 [==============================] - 14s 4ms/step - loss: 0.0484 - acc: 0.9898 - val_loss: 0.0868 - val_acc: 0.9661


In [511]:
new_text = ["wheat exhausted in hospital canteen", 
            "my neighbors house has crying voices. I think they need help"]

new_text = np.array(new_text, dtype=object)[:, np.newaxis]
with tf.Session() as session:
  K.set_session(session)
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  model.load_weights('./model.h5')  
  predicts = model.predict(new_text)


In [541]:

categories = np.unique(X_train.label)
predict_logits = predicts.argmax(axis=1)
predict_labels = [categories[logit] for logit in predict_logits]
dict=['abuse','communication','food','fuel','health','travel']
ANS=[dict[s] for s in predict_labels]
print(ANS)

['food', 'abuse']


In [544]:
np.around(predicts*100,decimals=1)

array([[ 0. ,  0. , 58.2,  0. , 41.8,  0. ],
       [86.8, 11. ,  0. ,  0.2,  1.8,  0. ]], dtype=float32)

In [ ]:
dict=['abuse','communication','food','fuel','health','travel']

In [ ]:
############ Combining both above cases ################3

In [547]:
new_text = ["wheat exhausted in hospital canteen and my neighbors house has crying voices. I think they need help",
           "GIVE ME OIL"]

new_text = np.array(new_text, dtype=object)[:, np.newaxis]
with tf.Session() as session:
  K.set_session(session)
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  model.load_weights('./model.h5')  
  predicts = model.predict(new_text)


In [548]:

categories = np.unique(X_train.label)
predict_logits = predicts.argmax(axis=1)
predict_labels = [categories[logit] for logit in predict_logits]
dict=['abuse','communication','food','fuel','health','travel']
ANS=[dict[s] for s in predict_labels]
print(ANS)

['abuse', 'fuel']


In [549]:
np.around(predicts*100,decimals=1)

array([[63. ,  1.2,  6.9,  8.5, 18.8,  1.6],
       [ 0. ,  0. ,  0.4, 99.5,  0.1,  0. ]], dtype=float32)

In [ ]:
########## Multilabel - Data Augumented - Universal sentense Encoder -  Keras Modelling ####################

In [ ]:
#################################################################

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
# import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

from tensorflow import keras
from keras.layers import Input, Embedding, LSTM, Dense, Lambda
from keras.models import Model
import numpy as np
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, Callback
from keras.optimizers import Adam


from keras import backend as K

session  = tf.Session()
K.set_session(session)
# session.run(tf.global_variables_initializer())
# session.run(tf.tables_initializer())



Using TensorFlow backend.


In [2]:
from sklearn.model_selection import train_test_split
from sklearn import datasets
import pandas as pd

df = pd.DataFrame(columns=['label','text'])
trainset = datasets.load_files(container_path = 'dataset_small_less_categories',encoding = 'UTF-8')

def get_dataframe(lines):
#     lines = filename.splitlines()
    data = []
    for i in range(0, len(lines)):
        label = lines[i][0]
    #     label = label.split(",")[0]
        text = ' '.join(lines[i][1:])
        text = re.sub(',','', text)
        data.append([label, text])
    df = pd.DataFrame(data, columns=['label', 'text'])
    df.label = df.label.astype('category')
    return df

for k in range(len(trainset.target)):
    labelled_sent= trainset.data[k].strip().split('\r\n')
    Label=[trainset.target[k]]*len(labelled_sent)
    labelled_emb = list(zip([s for s in Label], [s for s in labelled_sent]))
    df_train = get_dataframe(labelled_emb)
    df=pd.concat([df,df_train])
    df.reset_index(inplace=True, drop=True)

In [3]:
df

,label,text
0,2,Two people suffering from fever and cold
1,2,I need to take my father to hospital for dialy...
2,2,Equipment required at hospital. patients need ...
3,2,Need an ambulance urgently
4,2,accident in the area and arm broken need to go...
...,...,...
308,4,Leave the future benefits and look at the pres...
309,4,Lufthansa prepares to resume flights to Canada...
310,4,Mam I'm migrant worker of uttrakhand stuck in ...
311,4,Many travelers are still waiting to get back t...


In [4]:
#trainset.data[0], trainset.target[0]

In [5]:
trainset

{'data': ["Two people suffering from fever and cold\r\nI need to take my father to hospital for dialysis.\r\nEquipment required at hospital. patients need mask and sanitizers\r\nNeed an ambulance urgently\r\naccident in the area and arm broken, need to go to hospital\r\nThe delivery workers of essential items should always wear masks as they come in contact with hundreds of people. Government should provide masks for them.\r\nMy blood pressure is low.\r\nfood poisoning by eating old distributed food\r\nI am hungry from last three days. My glucose level is very low and bp is very high. Need help urgently.\r\nI was travelling from karnataka to tamil nadu and met with an accident. My bike is broken and I need medical assistance for my fractured leg.\r\nmy daughter ate expired medicine. I am unable to call any hospital.\r\nmy aunt is feeling uneasiness since morning. This can be a symptom of corona. Send check up team immediately.\r\nI read a news article and I suspect I have cancer. As a 

In [6]:
pd.set_option('display.max_colwidth', 200)
df

,label,text
0,2,Two people suffering from fever and cold
1,2,I need to take my father to hospital for dialysis.
2,2,Equipment required at hospital. patients need mask and sanitizers
3,2,Need an ambulance urgently
4,2,accident in the area and arm broken need to go to hospital
...,...,...
308,4,Leave the future benefits and look at the present situation.Nothing is benefiting air travellers for now.People are stranded in different places and requesting you everyday to resume domestic flig...
309,4,Lufthansa prepares to resume flights to Canada in June .We can't even travel province to province.
310,4,Mam I'm migrant worker of uttrakhand stuck in WB. I'm trying several times to communicate with nodal officer or local police but failed. Kndly plz help how can i reach you from salanpur WB Local p...
311,4,Many travelers are still waiting to get back to their homes jobs and daily routine.


In [7]:
import random
data=[]
length=len(df)
for i in range(length * 10):
    num1 = i % length
    num2 = num1
    while (num1 == num2):
        num2=random.randrange(length)
    data.append([df.label[num1],df.label[num2],df.text[num1] + ' ' + df.text[num2]])
    
df2 = pd.DataFrame(data, columns=['label1','label2', 'text'])


In [8]:
df2['label2'].value_counts()

1    693
4    678
0    653
2    588
3    518
Name: label2, dtype: int64

In [9]:
#X_train[X_train["label1"] == 0]

In [10]:
# df2.to_csv('dataset_multi.csv')

In [14]:
import spacy
import re

In [15]:
nlp = spacy.load("en_core_web_sm")
filter_POS = set(['NOUN', 'NUM', 'ADJ', 'VERB', 'PART'])
# ADV, NOUN, ADP, PRON, SCONJ, PROPN, DET, SYM, INTJ
# PUNCT, NUM, AUX, X, CONJ, ADJ, VERB, PART, SPACE, CCONJ
# def filter_query(query):
#     query = query.lower()
#     doc = nlp(query)
#     tokens = [t.text for t in doc if t.pos_ in filter_POS]
#     return ' '.join(tokens)

def filter_query(query):
    query = query.lower()
    # trans = str.maketrans(string.punctuation, ' '*len(string.punctuation))
    # query = query.translate(trans)
    # query = re.sub(r'\.', '. ', query)
    query = re.sub(r'[\t\n\r\f ]+', ' ', re.sub(r'\.', '. ', query))
    # print (query)
    doc = nlp(query)
    print ([(t.text,t.pos_) for t in doc])
    tokens = [t.text for t in doc if t.pos_ in filter_POS]
    tokens = [nlp(t)[0].lemma_ for t in tokens]
    return ' '.join(tokens)

In [16]:
query = 'ventilators and simcards. are unavialable'
query = query.lower()
query = re.sub(r'[\t\n\r\f ]+', ' ', re.sub(r'\.', '. ', query))
doc = nlp(query)
tokens = [t.text for t in doc if t.pos_ in filter_POS]
tokens = [nlp(t)[0].lemma_ for t in tokens]
print (tokens)

['ventilator', 'simcard', 'unavialable']


In [17]:
re.sub(r'\.', '. ', 'rice.wheat, masks, sim cards all are available as essential items')

'rice. wheat, masks, sim cards all are available as essential items'

In [18]:
re.sub(r'[\t\n\r\f ]+', ' ', re.sub(r'\.', '. ', 'gun    .is crap'))
# re.sub(r'\.', '. ', 'hi. hi')

'gun . is crap'

In [24]:
filter_query('rice, wheat, masks, sim cards all are available as essential items')
#filter_query('gun.is crap')

[('rice', 'NOUN'), (',', 'PUNCT'), ('wheat', 'NOUN'), (',', 'PUNCT'), ('masks', 'NOUN'), (',', 'PUNCT'), ('sim', 'NOUN'), ('cards', 'NOUN'), ('all', 'DET'), ('are', 'AUX'), ('available', 'ADJ'), ('as', 'SCONJ'), ('essential', 'ADJ'), ('items', 'NOUN')]


'rice wheat mask sim card available essential item'

In [13]:
df2['text_'] = df2['text']
df2['text'] = df2['text'].apply(filter_query)

# df['text_'] = df['text']
# df['text'] = df['text'].apply(filter_query)

In [14]:
df2

,label1,label2,text,text_
0,2,4,two people suffering fever cold want to go family place staying hotel,Two people suffering from fever and cold Want to go back to my family place. Currently staying in the hotel.
1,2,3,need to take father hospital dialysis love selfcare selflove trauma confused depression anxiety reactive abuse,I need to take my father to hospital for dialysis. love selfcare selflove trauma single4life confused depression anxiety reactive abuse
2,2,3,equipment required hospital patients need mask sanitizers media houses to much liberty harass people lot,Equipment required at hospital. patients need mask and sanitizers Media houses have to much of liberty and they harass people a lot.
3,2,1,need ambulance need prepared food 20 people slum,Need an ambulance urgently Need prepared food for 20 people in slum
4,2,2,accident area arm broken need to go hospital sparking electric generator fuel leakage got third degree burn,accident in the area and arm broken need to go to hospital There was sparking in my electric generator due to fuel leakage and I got a third degree burn.
...,...,...,...,...
3125,4,0,leave future benefits look present situation benefiting air travellers people stranded different places requesting to resume domestic flights week vodacom network ballons went dead,Leave the future benefits and look at the present situation.Nothing is benefiting air travellers for now.People are stranded in different places and requesting you everyday to resume domestic flig...
3126,4,4,prepares to resume flights can t travel province to province can e pass to travel prayagraj brother stuck lockdown veryuch trouble request government to help situation,Lufthansa prepares to resume flights to Canada in June .We can't even travel province to province. How can i get the e-pass to travel back from New Delhi to Prayagraj. My Brother's is Stuck in New...
3127,4,3,migrant worker stuck trying several times to communicate nodal officer local police failed help can reach local police said bring personal vehicle pass can issue requesting supplies,Mam I'm migrant worker of uttrakhand stuck in WB. I'm trying several times to communicate with nodal officer or local police but failed. Kndly plz help how can i reach you from salanpur WB Local p...
3128,4,1,many travelers waiting to homes jobs daily routine man could eating rice breakfast dinner tea,Many travelers are still waiting to get back to their homes jobs and daily routine. Man Could Soon Be Eating Rice For Breakfast Dinner And Tea.


In [35]:
X_train, X_test, y_train, y_test = train_test_split(df2, df2['label2'], test_size=0.2)
X_train.reset_index(inplace=True, drop=True)
X_test.reset_index(inplace=True, drop=True)
y_train.reset_index(inplace=True, drop=True)
y_test.reset_index(inplace=True, drop=True)


# X_train.reset_index(drop=True)
# X_test.reset_index(drop=True)
# y_train.reset_index(drop=True)
# y_test.reset_index(drop=True)

In [36]:
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np

main=[]
for i in range(len(X_train)):
    arr=np.array([X_train['label1'][i],X_train['label2'][i]])
    main.append(arr)
    
# main=[]
# for i in range(len(X_train)):
#     arr=np.array([X_train['label'][i]])
#     main.append(arr)
    
# # Create MultiLabelBinarizer object
# one_hot = MultiLabelBinarizer()

# One-hot encode data
train_text = np.array(X_train, dtype=object)[:,2][:,np.newaxis]
# train_label =np.asarray(one_hot.fit_transform(main), dtype = np.int8)

train_label = np.zeros((X_train.shape[0], len(trainset.target)))
for i in range(X_train.shape[0]):
    train_label[i][main[i]] = 1

In [37]:
train_label[9], X_train.loc[9]

(array([0., 1., 0., 0., 1.]),
 label1                                                                                                                                                                                                          1
 label2                                                                                                                                                                                                          4
 text                                                                                        need food hungry 70 workmen stuck project to go homes help nodal officer migrant workers take needful action earliest
 text_     Need Food Hungry 70 workmen stuck in Tidong HE Project near Recongpeo to go back to their homes in Bihar. No help from any nodal officer for migrant workers yet. Please take needful action at the ...
 Name: 9, dtype: object)

In [38]:
main[10],train_label[10], train_text[10]

(array([2, 4], dtype=int64),
 array([0., 0., 1., 0., 1.]),
 array(['hometown wuxicity not dispatched medical workers province hit covid19 donated medical supplies sister sities 16 countries appreciated travel registered nurse supposed to come county to work cancelled hospital starting help stuck'],
       dtype=object))

In [19]:
len(trainset.target)

5

In [39]:
main=[]
for i in range(len(X_test)):
    arr=np.array([X_test['label1'][i],X_test['label2'][i]])
    main.append(arr)
    
# main=[]
# for i in range(len(X_test)):
#     arr=np.array([X_test['label'][i]])
#     main.append(arr)
    
# # Create MultiLabelBinarizer object
# one_hot = MultiLabelBinarizer()

test_text = np.array(X_test, dtype=object)[:,2][:,np.newaxis]
# test_label = np.asarray(one_hot.fit_transform(main), dtype = np.int8)

test_label = np.zeros((X_test.shape[0], len(trainset.target)))
for i in range(X_test.shape[0]):
    test_label[i][main[i]] = 1

In [40]:
category_counts = len(np.unique(df.label))
module_url = "../tf_sent_encoder_2" 
embed = hub.Module(module_url)
embed_size = embed.get_output_info_dict()['default'].get_shape()[1].value

In [22]:
def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)),signature="default", as_dict=True)["default"]

# def sentence_encoder(input_text):
#     return embed(tf.squeeze(input_text))

In [23]:

input_text = Input(shape=(1,), dtype=tf.string)
embedding = Lambda(UniversalEmbedding,output_shape=(embed_size,))(input_text)
dense1 = Dense(1024, activation='relu')(embedding)
dense2 = Dense(512, activation='relu')(dense1)
dense3 = Dense(128, activation='relu')(dense2)
pred = Dense(category_counts, activation='sigmoid')(dense3)
model = Model(inputs=[input_text], outputs=pred)

LEARNING_RATE = 0.001

optimizer = Adam(lr=LEARNING_RATE)

model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

session.run(tf.global_variables_initializer())
session.run(tf.tables_initializer())



INFO:tensorflow:Saver not created because there are no variables in the graph to restore



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [24]:
# session.run(tf.global_variables_initializer())
# session.run(tf.tables_initializer())

# history = model.fit(train_text, 
#         train_label,
#         validation_data=(test_text, test_label),epochs=100,batch_size=256)
# model.save_weights('./model4.h5')


from keras.callbacks import EarlyStopping


class LearningRateTracker(Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(" - lr: {}".format(K.eval(self.model.optimizer.lr))) 
# session.run(tf.global_variables_initializer())
# session.run(tf.tables_initializer())

LR_PATIENCE = 10
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=LR_PATIENCE, min_lr=1e-8, verbose=1, mode="min")
es_callback = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)
lr_tracker = LearningRateTracker()

history = model.fit(train_text, 
          train_label,
          validation_data=(test_text, test_label),
          epochs=1000,
          batch_size=256,
          callbacks=[es_callback, lr_tracker, reduce_lr])
model.save_weights('./model5.h5')




Train on 2504 samples, validate on 626 samples
Epoch 1/1000




2504/2504 [==============================] - 9s 4ms/step - loss: 0.6071 - acc: 0.6657 - val_loss: 0.4687 - val_acc: 0.7760
 - lr: 0.0010000000474974513
Epoch 2/1000
2504/2504 [==============================] - 2s 696us/step - loss: 0.3696 - acc: 0.8498 - val_loss: 0.2741 - val_acc: 0.8923
 - lr: 0.0010000000474974513
Epoch 3/1000
2504/2504 [==============================] - 2s 690us/step - loss: 0.2572 - acc: 0.8982 - val_loss: 0.2513 - val_acc: 0.8994
 - lr: 0.0010000000474974513
Epoch 4/1000
2504/2504 [==============================] - 1s 571us/step - loss: 0.2223 - acc: 0.9126 - val_loss: 0.2152 - val_acc: 0.9121
 - lr: 0.0010000000474974513
Epoch 5/1000
2504/2504 [==============================] - 1s 573us/step - loss: 0.2007 - acc: 0.9219 - val_loss: 0.2034 - val_acc: 0.9208
 - lr: 0.0010000000474974513
Epoch 6/1000
2504/2504 [==============================] - 1s 492us/step - loss: 0.1777 - acc: 0.9326 - val_loss: 0

In [33]:
model.load_weights('./model5.h5')  

In [51]:
import csv
queries_list = []
with open('COVID-TEXT.csv', 'r', newline='') as f:
    reader = csv.reader(f)
    for row in reader:
        queries_list.append(row[0])

In [52]:
queries_list

['Labourers without food',
 'Migrant Workers from UP and BIHAR',
 'People need cooked food',
 'people stuck in bangalore',
 'People Stuck without Food',
 'People without ration',
 'Residence of rama swamy palya facing water scarcity',
 'Stranded migrants from various states',
 '5o Migrant workers from West Bengal',
 'Alemari samudaya with food and ration',
 'Elder people need urgent ration',
 'Familiy without ration',
 '10 members need Ration , Daily Wage workers',
 '10 people Need information on how to go back to bihar ',
 '10 person from Bihar are struck in Dharwad. They need Ration , Kindly deliver, they have no money and food to survive .\nThanks',
 '14 Workers need ration',
 '150 people Wants to go home from mangalore to Giridih Jarkhand',
 '17 families are not having ration card not received any ration from anywhere. They need to be helped for the basic food and sanitaries',
 '25 people who are living in Venkateshwar Nagar, (Vaddargalli) not ration card and not benefiting from PD

In [28]:
def run_prediction(query_list):
    filt_query = [filter_query(q) for q in query_list]
    print (filt_query)
    # filt_query = query_list
    query_arr = np.array(filt_query, dtype=object)[:, np.newaxis]
    predicts = model.predict(query_arr)
    return predicts

In [54]:
new_text = ["No network connectivity in my area for ordering groceries. Need ration urgently",
"Need gas cylinder as my mobile broke down.",
"My father is sick. i want daily ration",
"Yesterday i saw few people bullying a person from my locality. I think he needs help.i need cab as my car broke down to go to hospital.",
'I am unable to find fruits and vegetables']


# new_text = np.array(new_text, dtype=object)[:, np.newaxis]
# #new_text = np.array(queries_list, dtype=object)[:, np.newaxis]

# predicts = model.predict(new_text)
predicts = run_prediction(queries_list)

dictt=trainset.target_names

for j in range(len(new_text)):
    print("QUERY "+str(j) +'-> ' + new_text[j])
    print('\n')
    for i in range(len(dictt)):
        print('\t'+dictt[i]+":"+str(np.around(predicts*100,decimals=1)[j][i]))
    print('\n')

['labourers food', 'migrant workers bihar', 'people need cooked food', 'people stuck', 'people stuck food', 'people ration', 'facing water scarcity', 'stranded migrants various states', '5o migrant workers', 'food ration', 'elder people need urgent ration', 'familiy ration', '10 members need ration daily wage workers', '10 people need information to go', '10 person struck need ration deliver money food to survive thanks', '14 workers need ration', '150 people wants to go mangalore to', '17 families not having ration card not received ration need to helped basic food sanitaries', '25 people living not ration card not benefiting people need to helped provision kit', '3 members need ration gas', '3 migrants labourers need ration', '300 family poor people help', '300 migrants labourers need ration', '310 families need families', '32 family need ration daily wage workers', '6 members need ration garment workers', '6 members need ration wage workers daily wage workers', 'enquiry travelling',

In [55]:
result_df = pd.DataFrame([[queries_list[i]] + list(np.around(predicts*100,decimals=1)[i]) for i in range(len(queries_list))], columns=['query'] + dictt)

In [56]:
result_df.head()

,query,communication,food,healthcare,others,travel
0,Labourers without food,0.000000,100.000000,0.3,35.900002,0.000000
1,Migrant Workers from UP and BIHAR,0.000000,7.500000,95.5,7.500000,98.099998
2,People need cooked food,7.500000,98.900002,9.1,98.099998,0.000000
3,people stuck in bangalore,99.500000,0.100000,0.0,100.000000,0.100000
4,People Stuck without Food,57.200001,91.300003,3.1,99.699997,0.000000


In [57]:
result_df.to_csv('results.csv', index=False)

In [34]:
new_text = ["We have a plea from Sunkadkatte for rations. Request your urgent help .They receive food sometimes but not regularly. They have children and would prefer ration.",
"There is a Dalit village with 150 families in Kanakapura district who are not receiving any support. They received 10 kg rice and some wheat per family at the start of the lockdown.Anyone can help ?",
"Dear Ma'am Suchitra here again with another request for help. About 10 people and two children, tribals from Javadhi hills, Tamil Nadu working in coffee n pepper plantation in Chikmagalur. 3 of them spoke to me. They say over 150+ such labourers are there in this belt. They were paid by the contractor in March n have no money now. They have not been given any ration either. They met the Tasildar yesterday about going back. But that cannot happen now.  Ma'am how do we help them out? What is the state decision on migrant workers moving back to place of origin? Thank you.",
"Sir these wkrs r working as carpenters, helpers at Arabhikothanur Kolar,the builder and contractors r providing groceries and other food requirements to these workers.on enquiry the wkrs told there is no problem with resp to food sir",
"250 Fishermen From Andhra Pradesh struck in Udupi, Karnataka.They didn't have access to food from last few days. They are in depression.And at last he says, ""agar Kuch ni mila toh zehar kha lenge isse aacha .Please help.",
"Dear friends,  08 members from Uttar Pradesh are  in Mangalore near Kannanuru check post, they are in critical situation present they Requesting food and other help, any organizations working in Mangalore and border of Kerala request you pls attend this cae and help them, all are construction migrant workers,  their contact No: 6282662916, I hope some one will reach them. Please provide ration for all of us.",
"These are the contact details of a group of 50 plus migrant workers stuck in KR market with only intermittent access to food. They say they are a large group and all of them are unable to get food even at distribution points as food runs out. Request you to help",
'We are a small village on the outskirts of Bangalore and we do not have good cell reception here and hence we are isolated here with no news about coronavirus or availability of supplies.',
'Three construction workers from jharkhand are hungry from last two days and jumped off the building. Please provide ambulance',
'my friend fell from terrace while searching for network. There is network problem in this area. This is all government mistake and the medical expenses should be bourne by them.',
'The daily ration kit distributed to us do not have cooking oil in them. How are we supposed to cook.',
"I am trying to start a small kitchen to feed the hungry poor kids in my locality but I am unable to do so as I haven't received the big 19 kg gas cannister which I applied for almost 20 days ago.",
'I am a doctor, my car brokedown and want it to get repaired urgently.']
# new_text = np.array(new_text, dtype=object)[:, np.newaxis]


# predicts = model.predict(new_text)


# categories = np.unique(df.label)
# predict_logits = predicts.argmax(axis=1)
# predict_labels = [categories[logit] for logit in predict_logits]
predicts = run_prediction(new_text)

dictt=trainset.target_names
# ANS=[dict[s] for s in predict_labels]

for j in range(len(new_text)):
    print("QUERY "+str(j) +'-> ' + new_text[j])
    print('\n')
    for i in range(len(dictt)):
        print('\t'+dictt[i]+":"+str(np.around(predicts*100,decimals=1)[j][i]))
    print('\n')

['plea sunkadkatte rations request urgent help receive food not children would prefer ration', 'dalit village 150 families not receiving support received 10 kg rice wheat family start lockdown can help', 'request help 10 people two children tribals working coffee pepper plantation 3 spoke say 150 such labourers belt paid contractor money not given ration met tasildar yesterday going can not happen help state decision migrant workers moving place origin thank', 'working carpenters helpers builder contractors r providing groceries other food requirements workers enquiry wkrs told problem food sir', '250 fishermen struck didn access food last few days depression says help', 'dear friends 08 members mangalore near kannanuru check post critical situation present requesting food other help organizations working mangalore border request pls attend cae help construction migrant workers contact 6282662916 hope one will reach provide ration', 'contact details group 50 migrant workers stuck marke

In [38]:
new_text = ['I am unable to find fruits or vegetables',
'Where can I get bread',
'Edible supplies are not available in my area, i need bread and groceries',
'We are 6 people in my family and we are out of food. Anything you can give, please do',
'My inhaler isnt working and I need a new one asap, but i am unable to find it anywhere',
'I am running out of my insulin injections. I need to stock up',
'Me and my parents are stuck alone in our house with severe health issues. My father has heart problems and my mother is bedridden. I need urgent medical assistance',
'All electricity is down in my society due to which all electronic devices are off.Phone, wifi laptop all shut',
'I crashed in the divider on the way to medical and now bike wont start. Need help urgently.',
'Me and family stranded on roadside on the way to mysore. Car is not working',
'I hit the brakes very hard and now my car is shut down. Need help to get home',
'Need gas for car so that I can reach home.',
'Cooking oil is over and it is not available in any store',
'3 people have crashed in my neighbors house for some money matter. Need urgent assistance']
text = \
'''As my young daughter is still a toddler, I need some cereals or Cerelac and milk urgently, as she is almost always crying due to hunger.
Even though the local auhtorities have arranged for raw vegetables for us to have, please provide us with oil and spices to cook them, which we have been asking for so long.
I have been coughing for several days now, but recently I am also feeling a blockage on my throat, weaknesss and fever. 
My wife is a construction worker and since we don't have enough meals now due to shortage of money, she is weak and unable to get up
I came to Bangalore to work and have been stuck here for so long due to the lockdown, I need to get to my elderly parents who are in a small town in Maharashtra which is declared a red zone. 
My son went to Delhi to prepare for competitive exams and now he is stuck there with some friends as their institution has closed down and he is unable to return.
My neighbor, Mr. Rao is a salesman who is frustrated due to no income and he fights with his wife, shouts horribly and breaks things almost everyday
An elderly man in our locality was tested positive for coronavirus long back, has recovered and since then everybody is hateful of the entire family and even the shopkeepers harass them by refusing to sell items to them. 
We are a small village on the outskirts of Bangalore and we do not have good cell reception here and hence we are isolated here with no news about coronavirus or availability of supplies.
There has been transformer fault in our area and none of our devices are working and all our phones are discharged and we are not able to contact anyone to request for repair for many days now. 
We have been eating raw vegetables and fruits for the last 3 days as we do not have cooking gas in our cylinder and no one has delivered our new cylinder which we have requested a long time back.
I am trying to start a small kitchen to feed the hungry poor kids in my locality but I am unable to do so as I haven't received the big 19 kg gas cannister which I applied for almost 20 days ago. '''
new_text = text.split('\n')
print (new_text)

predicts = run_prediction(new_text)

dictt=['abuse','communication','food','fuel','health','travel']
# ANS=[dict[s] for s in predict_labels]

for j in range(len(new_text)):
    print("QUERY "+str(j) +'-> ' + new_text[j])
    print('\n')
    for i in range(len(dictt)):
        print('\t'+dictt[i]+":"+str(np.around(predicts*100,decimals=1)[j][i]))
    print('\n')

['As my young daughter is still a toddler, I need some cereals or Cerelac and milk urgently, as she is almost always crying due to hunger.', 'Even though the local auhtorities have arranged for raw vegetables for us to have, please provide us with oil and spices to cook them, which we have been asking for so long.', 'I have been coughing for several days now, but recently I am also feeling a blockage on my throat, weaknesss and fever. ', "My wife is a construction worker and since we don't have enough meals now due to shortage of money, she is weak and unable to get up", 'I came to Bangalore to work and have been stuck here for so long due to the lockdown, I need to get to my elderly parents who are in a small town in Maharashtra which is declared a red zone. ', 'My son went to Delhi to prepare for competitive exams and now he is stuck there with some friends as their institution has closed down and he is unable to return.', 'My neighbor, Mr. Rao is a salesman who is frustrated due to 

In [ ]:
["We have a plea from Sunkadkatte for rations. Request your urgent help .They receive food sometimes but not regularly. They have children and would prefer ration.",
"There is a Dalit village with 150 families in Kanakapura district who are not receiving any support. They received 10 kg rice and some wheat per family at the start of the lockdown.Anyone can help ?",
"Dear Ma'am Suchitra here again with another request for help. About 10 people and two children, tribals from Javadhi hills, Tamil Nadu working in coffee n pepper plantation in Chikmagalur. 3 of them spoke to me. They say over 150+ such labourers are there in this belt. They were paid by the contractor in March n have no money now. They have not been given any ration either. They met the Tasildar yesterday about going back. But that cannot happen now.  Ma'am how do we help them out? What is the state decision on migrant workers moving back to place of origin? Thank you.",
"Sir these wkrs r working as carpenters, helpers at Arabhikothanur Kolar,the builder and contractors r providing groceries and other food requirements to these workers.on enquiry the wkrs told there is no problem with resp to food sir",
"250 Fishermen From Andhra Pradesh struck in Udupi, Karnataka.They didn't have access to food from last few days. They are in depression.And at last he says, ""agar Kuch ni mila toh zehar kha lenge isse aacha .Please help.",
"Dear friends,  08 members from Uttar Pradesh are  in Mangalore near Kannanuru check post, they are in critical situation present they Requesting food and other help, any organizations working in Mangalore and border of Kerala request you pls attend this cae and help them, all are construction migrant workers,  their contact No: 6282662916, I hope some one will reach them.",
"These are the contact details of a group of 50 plus migrant workers stuck in KR market with only intermittent access to food. They say they are a large group and all of them are unable to get food even at distribution points as food runs out. Request you to help"]

In [33]:
predicts.shape
preds_round = np.around(predicts*100,decimals=1)

In [69]:
from statistics import stdev
pred_results = []
for i in range(predicts.shape[0]):
    sorted_class_similariy = [float(x) for x in sorted(preds_round[i], reverse=True)]
    print (sorted_class_similariy)
    split_stdev = []
    split_stdev.append(stdev(sorted_class_similariy[1:len(dictt)]))
    for k in range(2, len(dictt) - 1):
        split_stdev.append((stdev(sorted_class_similariy[:k]) + stdev(sorted_class_similariy[k:]))/2)
    split_stdev.append(stdev(sorted_class_similariy[:len(dictt) - 1]))

    split_ind = np.argmin(split_stdev)
    thres = (sorted_class_similariy[split_ind] + sorted_class_similariy[split_ind + 1]) / 2
    # print (thres)
    pred_labels = [c for c in range(len(dictt)) if preds_round[i][c] >= np.around(thres, decimals=1)]
    pred_classes = [dictt[r] for r in pred_labels]
    # print (pred_labels)
    pred_results.append(pred_classes)

[100.0, 92.9000015258789, 0.5, 0.0, 0.0, 0.0]
[100.0, 6.800000190734863, 5.5, 0.20000000298023224, 0.0, 0.0]
[99.9000015258789, 83.80000305175781, 56.900001525878906, 0.0, 0.0, 0.0]
[94.80000305175781, 93.80000305175781, 35.099998474121094, 2.200000047683716, 0.8999999761581421, 0.0]
[100.0, 70.9000015258789, 0.5, 0.0, 0.0, 0.0]
[100.0, 71.9000015258789, 0.0, 0.0, 0.0, 0.0]
[97.80000305175781, 41.79999923706055, 20.0, 10.0, 0.0, 0.0]
[100.0, 99.9000015258789, 3.200000047683716, 0.30000001192092896, 0.10000000149011612, 0.0]
[100.0, 4.199999809265137, 2.4000000953674316, 0.6000000238418579, 0.20000000298023224, 0.0]
[100.0, 100.0, 0.10000000149011612, 0.10000000149011612, 0.0, 0.0]
[100.0, 51.099998474121094, 2.299999952316284, 0.0, 0.0, 0.0]
[100.0, 27.5, 0.20000000298023224, 0.0, 0.0, 0.0]
[100.0, 2.200000047683716, 0.4000000059604645, 0.4000000059604645, 0.20000000298023224, 0.0]
[99.9000015258789, 6.5, 3.4000000953674316, 0.5, 0.20000000298023224, 0.10000000149011612]
[99.8000030517

In [70]:
pred_results

[['food', 'health'],
 ['food'],
 ['food', 'fuel', 'health'],
 ['communication', 'travel'],
 ['food', 'health'],
 ['food', 'health'],
 ['travel'],
 ['food', 'travel'],
 ['food'],
 ['food', 'health'],
 ['abuse', 'food'],
 ['food'],
 ['food'],
 ['food'],
 ['food', 'travel'],
 ['food'],
 ['communication', 'food'],
 ['food', 'travel'],
 ['food', 'health', 'travel'],
 ['food'],
 ['food', 'travel'],
 ['food'],
 ['food'],
 ['food'],
 ['food'],
 ['food', 'health'],
 ['food'],
 ['travel'],
 ['food', 'health', 'travel'],
 ['food', 'fuel', 'travel'],
 ['health'],
 ['food'],
 ['travel'],
 ['food'],
 ['food'],
 ['food'],
 ['food', 'health'],
 ['food'],
 ['travel'],
 ['communication', 'travel'],
 ['food'],
 ['food'],
 ['food', 'travel'],
 ['food'],
 ['food', 'travel'],
 ['food', 'fuel', 'health'],
 ['food', 'fuel', 'travel'],
 ['fuel'],
 ['food', 'fuel', 'travel'],
 ['food'],
 ['food'],
 ['abuse'],
 ['food', 'fuel'],
 ['travel'],
 ['food'],
 ['food', 'travel'],
 ['abuse', 'food'],
 ['abuse', 'fuel', 

In [ ]:
OK### both tanh -84% split=0.3    split=o.2 good
## both relu - 86% split=0.2
## both tanh - 85.4% split=0.2
## tanh relu 84.73
### relu tanh 85.63 -----good 256,128
### relu tanh  86.12-----ok ok 1024,256
### relu tanh  86.12-----ok ok 1024,256
## tanh tanh ---1024 128 
## relu tanh tanh 256 128 64- 85.8%
## tanh tanh 0.3 split acc= 84.55 can try


OK###with categorical cross entropy tanh tanh 256 128 results are ok but accuracy showing is less




### 93.82%
# dense1 = Dense(256, activation='relu')(embedding)
# dense2 = Dense(128, activation='relu')(dense1)
# pred = Dense(category_counts, activation='sigmoid')(dense2)
# model = Model(inputs=[input_text], outputs=pred)
# model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])




### 94.41%
# dense1 = Dense(1024, activation='relu')(embedding)
# dense2 = Dense(256, activation='relu')(dense1)
# pred = Dense(category_counts, activation='sigmoid')(dense2)
# model = Model(inputs=[input_text], outputs=pred)
# model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])



## __94.78____%
# dense1 = Dense(1024, activation='relu')(embedding)
# dense2 = Dense(128, activation='relu')(dense1)
# pred = Dense(category_counts, activation='sigmoid')(dense2)
# model = Model(inputs=[input_text], outputs=pred)
# model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])



####### 94.48 % #########

# input_text = Input(shape=(1,), dtype=tf.string)
# embedding = Lambda(UniversalEmbedding,output_shape=(embed_size,))(input_text)
# dense1 = Dense(1024, activation='relu')(embedding)
# dense2 = Dense(128, activation='tanh')(dense1)
# pred = Dense(category_counts, activation='sigmoid')(dense2)
# model = Model(inputs=[input_text], outputs=pred)
# model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])

In [41]:
categories = trainset.target_names
categories

['communication', 'food', 'healthcare', 'others', 'travel']

In [42]:
from keras.layers import Concatenate

def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)),signature="default", as_dict=True)["default"]

# def sentence_encoder(input_text):
#     return embed(tf.squeeze(input_text))

input_text = Input(shape=(1,), dtype=tf.string)
embedding = Lambda(UniversalEmbedding,output_shape=(embed_size,))(input_text)
dense1 = Dense(1024, activation='relu')(embedding)
dense2 = Dense(128, activation='relu')(dense1)
pred = []
for c in categories:
    pred.append(Dense(1, activation='sigmoid')(dense2))
out = Concatenate()(pred)
model = Model(inputs=[input_text], outputs=out)

LEARNING_RATE = 0.001

optimizer = Adam(lr=LEARNING_RATE)

model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

session.run(tf.global_variables_initializer())
session.run(tf.tables_initializer())

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [43]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, 512)          0           input_3[0][0]                    
__________________________________________________________________________________________________
dense_15 (Dense)                (None, 1024)         525312      lambda_3[0][0]                   
__________________________________________________________________________________________________
dense_16 (Dense)                (None, 128)          131200      dense_15[0][0]                   
__________________________________________________________________________________________________
dense_17 (

In [44]:


from keras.callbacks import EarlyStopping


class LearningRateTracker(Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(" - lr: {}".format(K.eval(self.model.optimizer.lr))) 
# session.run(tf.global_variables_initializer())
# session.run(tf.tables_initializer())

LR_PATIENCE = 10
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=LR_PATIENCE, min_lr=1e-8, verbose=1, mode="min")
es_callback = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)
lr_tracker = LearningRateTracker()

history = model.fit(train_text, 
          train_label,
          validation_data=(test_text, test_label),
          epochs=1000,
          batch_size=256,
          callbacks=[es_callback, lr_tracker, reduce_lr])
model.save_weights('./model5.h5')

Train on 2504 samples, validate on 626 samples
Epoch 1/1000
2504/2504 [==============================] - 26s 10ms/step - loss: 0.6200 - acc: 0.6868 - val_loss: 0.5276 - val_acc: 0.7329
 - lr: 0.0010000000474974513
Epoch 2/1000
2504/2504 [==============================] - 2s 659us/step - loss: 0.4316 - acc: 0.8216 - val_loss: 0.3591 - val_acc: 0.8677
 - lr: 0.0010000000474974513
Epoch 3/1000
2504/2504 [==============================] - 2s 750us/step - loss: 0.3007 - acc: 0.8831 - val_loss: 0.2829 - val_acc: 0.8856
 - lr: 0.0010000000474974513
Epoch 4/1000
2504/2504 [==============================] - 2s 624us/step - loss: 0.2456 - acc: 0.9020 - val_loss: 0.2569 - val_acc: 0.8949
 - lr: 0.0010000000474974513
Epoch 5/1000
2504/2504 [==============================] - 1s 586us/step - loss: 0.2222 - acc: 0.9093 - val_loss: 0.2485 - val_acc: 0.8952
 - lr: 0.0010000000474974513
Epoch 6/1000
2504/2504 [==============================] - 2s 642us/step - loss: 0.2065 - acc: 0.9172 - val_loss: 0.231

In [ ]:
categories = trainset.target_names
categories

from keras.layers import Concatenate

def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)),signature="default", as_dict=True)["default"]

# def sentence_encoder(input_text):
#     return embed(tf.squeeze(input_text))

input_text = Input(shape=(1,), dtype=tf.string)
embedding = Lambda(UniversalEmbedding,output_shape=(embed_size,))(input_text)
dense1 = Dense(1024, activation='relu')(embedding)
dense2 = Dense(128, activation='relu')(dense1)
pred = []
for c in categories:
    pred.append(Dense(1, activation='sigmoid')(dense2))
out = Concatenate()(pred)
model = Model(inputs=[input_text], outputs=out)

LEARNING_RATE = 0.001

optimizer = Adam(lr=LEARNING_RATE)

model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

session.run(tf.global_variables_initializer())
session.run(tf.tables_initializer())

In [93]:
import nltk
new_text = ['I am unable to find fruits or vegetables',
'Where can I get bread',
'Edible supplies are not available in my area, i need bread and groceries',
'We are 6 people in my family and we are out of food. Anything you can give, please do',
'My inhaler isnt working and I need a new one asap, but i am unable to find it anywhere',
'I am running out of my insulin injections. I need to stock up',
'Me and my parents are stuck alone in our house with severe health issues. My father has heart problems and my mother is bedridden. I need urgent medical assistance',
'All electricity is down in my society due to which all electronic devices are off.Phone, wifi laptop all shut',
'I crashed in the divider on the way to medical and now bike wont start. Need help urgently.',
'Me and family stranded on roadside on the way to mysore. Car is not working',
'I hit the brakes very hard and now my car is shut down. Need help to get home',
'Need gas for car so that I can reach home.',
'Cooking oil is over and it is not available in any store',
'3 people have crashed in my neighbors house for some money matter. Need urgent assistance']

In [98]:
ind = 1
nltk.pos_tag(nltk.word_tokenize('i want to go back'))

[('i', 'NN'), ('want', 'VBP'), ('to', 'TO'), ('go', 'VB'), ('back', 'RB')]

In [96]:
filter_query(new_text[ind])

[('where', 'ADV'), ('can', 'VERB'), ('i', 'PRON'), ('get', 'AUX'), ('bread', 'NOUN')]


'can bread'

In [62]:
from nltk.stem import WordNetLemmatizer

In [63]:
ps = PorterStemmer()
lemmatizer = WordNetLemmatizer() 

In [64]:
words = ["program", "programs", "computer", "programing", "programers"] 
for w in words: 
    print(w, " : ", ps.stem(w)) 

program  :  program
programs  :  program
computer  :  comput
programing  :  program
programers  :  program


In [89]:
print("rocks :", lemmatizer.lemmatize("rocks")) 
print(lemmatizer.lemmatize("cards")) 

# a denotes adjective in "pos" 
print("better :", lemmatizer.lemmatize("better", pos ="a"))

rocks : rock
card
better : good
